In [1]:
from pymongo import MongoClient
from pymongo import errors
from pprint import pprint

In [2]:
def write_to_crawledPerfumes(data):
    client = MongoClient('localhost', 27017)
    Perfume_Database = client.Perfume_Database
    crawled_perfumes = Perfume_Database["Crawled_Perfumes"]
    crawled_perfumes.insert_many(data)

In [3]:
def readall_CleanedCrawledPerfumes(query):
    client = MongoClient('localhost', 27017)
    Perfume_Database = client["Perfume_Database"]
    crawled_perfumes = Perfume_Database["Cleared_Crawled_Perfumes"]
    data = crawled_perfumes.find(query)
    
    return data

In [4]:
def perfume_exists(url):
    client = MongoClient('localhost', 27017)
    Perfume_Database = client["Perfume_Database"]
    Scraped_Perfumes = Perfume_Database["Scraped_Perfumes"]
    
    res = Scraped_Perfumes.find_one({"url": url})
    
    if res:
        return True
    else: 
        return False

In [5]:
def write_one_to_ScrapedPerfumes(perfume):
    client = MongoClient('localhost', 27017)
    Perfume_Database = client.Perfume_Database
    Scraped_Perfumes = Perfume_Database["Scraped_Perfumes"]
    Scraped_Perfumes.insert_one(perfume)

In [6]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium
from pandas import DataFrame
import json
from selenium.webdriver.chrome.options import Options
import random

Create a new profile path, and manually disable site data saving so fragrantica won't ask for cloudflare tests and also cookies.

In [7]:
def init_browser(proxy=''): # MUST add settings to prevent chrome from saving site data
    chrome_options = Options()
    chrome_options.add_argument("user-data-dir=B:\Projects\Perfume-DataScrape\Chrome");
    if proxy:
        chrome_options.add_argument('--proxy-server=' + proxy)
    
    return Browser("chrome", options=chrome_options, headless=False)

In [8]:
def scrape_perfume(browser):
    soup = bs(browser.html, "html.parser")

## Scrape the perfume page for Name, Company, Main Accords, Perfumer, Notes

In [9]:
def scrape_perfume_page(browser, url):
    
####################################################
####### USE THIS FOR A SINGULAR PERFUME PAGE #######
############### NOT THE SEARCH PAGE ###############
###################################################
    
    soup = bs(browser.html, "html.parser")

    time.sleep(1)

    ####### PULL FROM ORIGINAL NAME LIST #######
    try:
        perfume_name = soup.find_all("div", class_="cell small-12")[3].find_all("b")[0].get_text()
    except:
        perfume_name = "NA"
        
    try:
        perfume_comp = soup.find_all("div", class_="cell small-12")[3].find_all("b")[1].get_text()
    except:
        perfume_comp = "NA"
        
    try:
        perfume_image = soup.find_all("div", class_="cell small-12")[1].find("img")["src"]
    except:
        perfume_image = "NA"

    try: 
        for_gender = soup.find("small").get_text()
    except:
        for_gender = "NA"
    
    try:
        rating = float(soup.find("p", class_="info-note").find_all("span")[0].get_text())
        number_votes = int(soup.find("p", class_="info-note").find_all("span")[2].get_text().replace(',', ''))
    except:
        rating = "NA"
        number_votes = "NA"
        print(f"{perfume_name} does not have a ranking")

    try:
        description = soup.find_all("div", class_="cell small-12")[3].get_text()
    except:
        description = "NA"
        print(f"{perfume_name} does not have a description")

    ####### MAIN ACCORDS DICTIONARY #######

    try:
        main_accords = soup.find_all("div", class_="cell accord-box")
        accords_dict = {}
        for m in range(len(main_accords)):
            accord_name = main_accords[m].get_text()
            accord_value = float(main_accords[m].find("div", class_="accord-bar")["style"].rsplit("width: ")[1].strip("%;"))
            accords_dict[accord_name] = accord_value
    except:
        accords_dict = {}
        print(f"{perfume_name} does not have accords")

    ####### FRAGRANCE NOTES #######        
    notes = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; text-align: center; flex-flow: wrap; align-items: flex-end; padding: 0.5rem;"})

    if len(notes) == 3:
        number = 2
        top_notes_list = []
        middle_notes_list = []
        base_notes_list = []

        for n in range(len(notes[0].find_all("span", class_="link-span"))):
            top_notes_list.append(notes[0].find_all("div")[number].get_text())
            number += 3

        number = 2
        for p in range(len(notes[1].find_all("span", class_="link-span"))):
            middle_notes_list.append(notes[1].find_all("div")[number].get_text())
            number += 3

        number = 2
        for q in range(len(notes[2].find_all("span", class_="link-span"))):
            base_notes_list.append(notes[2].find_all("div")[number].get_text())
            number += 3
    elif len(notes) == 2:
        number = 2
        top_notes_list = []
        middle_notes_list = []
        base_notes_list = []

        for r in range(len(notes[0].find_all("span", class_="link-span"))):
            top_notes_list.append(notes[0].find_all("div")[number].get_text())
            number += 3

        number = 2
        for s in range(len(notes[1].find_all("span", class_="link-span"))):
            middle_notes_list.append(notes[1].find_all("div")[number].get_text())
            number += 3
    elif len(notes) == 1:
        number = 2
        top_notes_list = []
        middle_notes_list = []
        base_notes_list = []

        for v in range(len(notes[0].find_all("span", class_="link-span"))):
            middle_notes_list.append(notes[0].find_all("div")[number].get_text())
            number += 3
    else:
        top_notes_list = []
        middle_notes_list = []
        base_notes_list = []
    
    ####### VOTING DATA & INFORMATION #######
    try:
        voting = soup.find_all("div", class_="cell small-1 medium-1 large-1")
    except:
        voting = "NA"
        
    ####### Longevity #######
    try:
        long_v_weak = int(voting[0].get_text())
    except:
        long_v_weak = "NA"
        
    try:
        long_weak = int(voting[1].get_text())
    except:
        long_weak = "NA"
        
    try:
        long_moderate = int(voting[2].get_text())
    except:
        long_moderate = "NA"
        
    try:
        long_long_last = int(voting[3].get_text())
    except:
        long_long_last = "NA"
    
    try:
        long_eternal = int(voting[4].get_text())
    except:
        long_eternal = "NA"
    
    
    ####### Sillage #######
    try:
        sill_intimate = int(voting[5].get_text())
    except:
        sill_intimate = "NA"
        
    try:
        sill_intimate = int(voting[5].get_text())
    except:
        sill_intimate = "NA"
        
    try:
        sill_moderate = int(voting[6].get_text())
    except:
        sill_moderate = "NA"
        
    try:
        sill_strong = int(voting[7].get_text())
    except:
        sill_strong = "NA"
        
    try:
        sill_enormus = int(voting[8].get_text())
    except:
        sill_enormus = "NA"
        
    ####### Gender #######
    try:
        gender_female = int(voting[9].get_text())
    except:
        gender_female = "NA"
        
    try:
        gender_more_fem = int(voting[10].get_text())
    except:
        gender_more_fem = "NA"
        
    try:
        gender_unisex = int(voting[11].get_text())
    except:
        gender_unisex = "NA"
        
    try:
        gender_more_male = int(voting[12].get_text())
    except:
        gender_more_male = "NA"
        
    try:
        gender_male = int(voting[13].get_text())
    except:
        gender_male = "NA"
        
    ####### Price Value #######
    try:
        value_w_over = int(voting[14].get_text())
    except:
        value_w_over = "NA"
        
    try:
        value_over = int(voting[15].get_text())
    except:
        value_over = "NA"
        
    try:
        value_ok = int(voting[16].get_text())
    except:
        value_ok = "NA"
        
    try:
        value_good = int(voting[17].get_text())
    except:
        value_good = "NA"
        
    try:
        value_great = int(voting[18].get_text())
    except:
        value_great = "NA"  

    ####### CREATING THE DICTIONARY OF DATA #######
    perfume_dict = {"url": url,
                    "name": perfume_name,
                    "company": perfume_comp,
                    "image": perfume_image,
                    "gender": for_gender,
                    "rating": rating,
                    "number_votes": number_votes,
                    "main accords": accords_dict,
                    "description": description.replace("Read about this perfume in other languages: Deutsch, Español, Français, Čeština, Italiano, Русский, Polski, Português, Ελληνικά, 汉语, Nederlands, Srpski, Română, العربية, Українська, Монгол, עברית", ''),
                    "top notes": top_notes_list,
                    "middle notes": middle_notes_list,
                    "base notes": base_notes_list,
                    "longevity":   {"very weak": long_v_weak,
                                    "weak": long_weak,
                                    "moderate": long_moderate,
                                    "long lasting": long_long_last,
                                    "eternal": long_eternal},
                    "sillage":     {"intimate": sill_intimate,
                                    "moderate": sill_moderate,
                                    "strong": sill_strong,
                                    "enormous": sill_enormus},
                    "gender_vote": {"female": gender_female,
                                    "more female": gender_more_fem,
                                    "unisex": gender_unisex,
                                    "more male": gender_more_male,
                                    "male": gender_male},
                    "price value": {"way overpriced": value_w_over,
                                    "overpriced": value_over,
                                    "ok": value_ok,
                                    "good value": value_good,
                                    "great value": value_great}
                   }

        
    return perfume_dict

In [19]:
def scrape_CrawledPerfumes(browser):
    counter = 0
    cooler = 120
    crawled_perfumes_cursor = readall_CleanedCrawledPerfumes({}).limit(5000)
    
    while True:
        try:
            crawled_perfume = crawled_perfumes_cursor.next()
            
            if not perfume_exists(crawled_perfume["link"]):
                
                browser.visit(crawled_perfume["link"])
                
                if am_I_blocked(browser):
                    print("\nI am BLOCKED")
                    print(f"Waiting {cooler} seconds ...\n")
                    supreme_wait(browser, cooler)
                    browser.visit(crawled_perfume["link"])
                    
                    while am_I_blocked(browser):
                        cooler = cooler * 2
                        print("\nI am BLOCKED")
                        print(f"Waiting {cooler} seconds ...\n")
                        supreme_wait(browser, cooler)
                        
                        browser.visit(crawled_perfume["link"])
                    else: 
                        cooler = 120
            
#               check if cloudflare is activated on the website
                elif is_cloudflared(browser):
                    raise Exception("Cloudflare has been activated.")
            
#               Check if the page is a perfume page, if not continue and ignore this one
                elif not is_perfume_page(browser):
                    raise Exception("This is not a perfume page appearently, check connection or link.")
            
            
                else:
                    perfume = scrape_perfume_page(browser, crawled_perfume['link'])
                    write_one_to_ScrapedPerfumes(perfume)
                    counter = counter +1

                    print(f"{counter}- Scraped and Saved {perfume['name']}.")

        except StopIteration:
            print("\n")
            print("==========================")
            print("Results are FINISHED.")
            break
            
        except Exception as e: 
            print(e.Message)
            continue

In [11]:
def am_I_blocked(browser):
    return browser.is_text_present("429 Too Many Requests")

In [12]:
def is_perfume_page(browser):
    the_text = "Read about this perfume in other languages: Deutsch, Español, Français, Čeština, Italiano, Русский, Polski, Português, Ελληνικά, 汉语, Nederlands, Srpski, Română, العربية, Українська, Монгол, עברית."
    return browser.is_text_present(the_text)

In [32]:
def supreme_wait(browser, cooler):
    for fan in range(cooler):
        
        if not is_perfume_page(browser):
            time.sleep(1)
            print(''.join('|' if i < fan*28/cooler else '-' for i in range(28)) + '|', end='\r')
        else:
            print(''.join(' ' for i in range(30)))
            break
            
    print(''.join(' ' for i in range(30)))

In [14]:
def is_cloudflared(browser):
    
    cloudflare_text = "www.fragrantica.com needs to review the security of your connection before proceeding."
    return browser.is_text_present(cloudflare_text)

In [18]:
is_cloudflared(browser)

False

In [15]:
def is_disconnected(browser):
    disconnected_text = "This site can’t be reached"
    return browser.is_text_present(disconnected_text)

In [21]:
browser = init_browser()

In [ ]:
scrape_CrawledPerfumes(browser)

1- Scraped and Saved No. 08 L’Eau Baptiste.
2- Scraped and Saved Ivy League.
3- Scraped and Saved Amber Elixir.
4- Scraped and Saved Free Wave For Her.
5- Scraped and Saved Mukhallat Dahn Al Oudh Moattaq.
6- Scraped and Saved XJ 1861 Decas.
7- Scraped and Saved Angelic Elixir.
8- Scraped and Saved Domenico Caraceni 1913.
9- Scraped and Saved Evasion Digo de Havane.
10- Scraped and Saved No.04 Bois de Balincourt.

I am BLOCKED
Waiting 120 seconds ...

                              
11- Scraped and Saved Anubis Kiss.
12- Scraped and Saved Vodka Love.
13- Scraped and Saved Heroic Man.
14- Scraped and Saved Black Aoud Intense.
15- Scraped and Saved Tam Dao Eau de Parfum.
16- Scraped and Saved First Pour l'Ete.
17- Scraped and Saved Acqua di Gio.
18- Scraped and Saved ART Nianse.
19- Scraped and Saved Cherry Blossom.

I am BLOCKED
Waiting 120 seconds ...

                              
20- Scraped and Saved Jolie.
21- Scraped and Saved Eden.
22- Scraped and Saved Musc Noir Rose For Her.
23-

Message: unknown error: net::ERR_PROXY_CONNECTION_FAILED
  (Session info: chrome=120.0.6099.224)
Stacktrace:
	GetHandleVerifier [0x00007FF66CA42142+3514994]
	(No symbol) [0x00007FF66C660CE2]
	(No symbol) [0x00007FF66C5076AA]
	(No symbol) [0x00007FF66C4FFB39]
	(No symbol) [0x00007FF66C4F25DD]
	(No symbol) [0x00007FF66C4F16FD]
	(No symbol) [0x00007FF66C4F0746]
	(No symbol) [0x00007FF66C4F06EB]
	(No symbol) [0x00007FF66C4EEE3D]
	(No symbol) [0x00007FF66C4EF603]
	(No symbol) [0x00007FF66C50A0FD]
	(No symbol) [0x00007FF66C592E01]
	(No symbol) [0x00007FF66C575FEA]
	(No symbol) [0x00007FF66C5928F6]
	(No symbol) [0x00007FF66C575D93]
	(No symbol) [0x00007FF66C544BDC]
	(No symbol) [0x00007FF66C545C64]
	GetHandleVerifier [0x00007FF66CA6E16B+3695259]
	GetHandleVerifier [0x00007FF66CAC6737+4057191]
	GetHandleVerifier [0x00007FF66CABE4E3+4023827]
	GetHandleVerifier [0x00007FF66C7904F9+689705]
	(No symbol) [0x00007FF66C66C048]
	(No symbol) [0x00007FF66C668044]
	(No symbol) [0x00007FF66C6681C9]
	(No s

Message: unknown error: net::ERR_PROXY_CONNECTION_FAILED
  (Session info: chrome=120.0.6099.224)
Stacktrace:
	GetHandleVerifier [0x00007FF66CA42142+3514994]
	(No symbol) [0x00007FF66C660CE2]
	(No symbol) [0x00007FF66C5076AA]
	(No symbol) [0x00007FF66C4FFB39]
	(No symbol) [0x00007FF66C4F25DD]
	(No symbol) [0x00007FF66C4F16FD]
	(No symbol) [0x00007FF66C4F0746]
	(No symbol) [0x00007FF66C4F06EB]
	(No symbol) [0x00007FF66C4EEE3D]
	(No symbol) [0x00007FF66C4EF603]
	(No symbol) [0x00007FF66C50A0FD]
	(No symbol) [0x00007FF66C592E01]
	(No symbol) [0x00007FF66C575FEA]
	(No symbol) [0x00007FF66C5928F6]
	(No symbol) [0x00007FF66C575D93]
	(No symbol) [0x00007FF66C544BDC]
	(No symbol) [0x00007FF66C545C64]
	GetHandleVerifier [0x00007FF66CA6E16B+3695259]
	GetHandleVerifier [0x00007FF66CAC6737+4057191]
	GetHandleVerifier [0x00007FF66CABE4E3+4023827]
	GetHandleVerifier [0x00007FF66C7904F9+689705]
	(No symbol) [0x00007FF66C66C048]
	(No symbol) [0x00007FF66C668044]
	(No symbol) [0x00007FF66C6681C9]
	(No s

Message: unknown error: net::ERR_PROXY_CONNECTION_FAILED
  (Session info: chrome=120.0.6099.224)
Stacktrace:
	GetHandleVerifier [0x00007FF66CA42142+3514994]
	(No symbol) [0x00007FF66C660CE2]
	(No symbol) [0x00007FF66C5076AA]
	(No symbol) [0x00007FF66C4FFB39]
	(No symbol) [0x00007FF66C4F25DD]
	(No symbol) [0x00007FF66C4F16FD]
	(No symbol) [0x00007FF66C4F0746]
	(No symbol) [0x00007FF66C4F06EB]
	(No symbol) [0x00007FF66C4EEE3D]
	(No symbol) [0x00007FF66C4EF603]
	(No symbol) [0x00007FF66C50A0FD]
	(No symbol) [0x00007FF66C592E01]
	(No symbol) [0x00007FF66C575FEA]
	(No symbol) [0x00007FF66C5928F6]
	(No symbol) [0x00007FF66C575D93]
	(No symbol) [0x00007FF66C544BDC]
	(No symbol) [0x00007FF66C545C64]
	GetHandleVerifier [0x00007FF66CA6E16B+3695259]
	GetHandleVerifier [0x00007FF66CAC6737+4057191]
	GetHandleVerifier [0x00007FF66CABE4E3+4023827]
	GetHandleVerifier [0x00007FF66C7904F9+689705]
	(No symbol) [0x00007FF66C66C048]
	(No symbol) [0x00007FF66C668044]
	(No symbol) [0x00007FF66C6681C9]
	(No s

Message: unknown error: net::ERR_PROXY_CONNECTION_FAILED
  (Session info: chrome=120.0.6099.224)
Stacktrace:
	GetHandleVerifier [0x00007FF66CA42142+3514994]
	(No symbol) [0x00007FF66C660CE2]
	(No symbol) [0x00007FF66C5076AA]
	(No symbol) [0x00007FF66C4FFB39]
	(No symbol) [0x00007FF66C4F25DD]
	(No symbol) [0x00007FF66C4F16FD]
	(No symbol) [0x00007FF66C4F0746]
	(No symbol) [0x00007FF66C4F06EB]
	(No symbol) [0x00007FF66C4EEE3D]
	(No symbol) [0x00007FF66C4EF603]
	(No symbol) [0x00007FF66C50A0FD]
	(No symbol) [0x00007FF66C592E01]
	(No symbol) [0x00007FF66C575FEA]
	(No symbol) [0x00007FF66C5928F6]
	(No symbol) [0x00007FF66C575D93]
	(No symbol) [0x00007FF66C544BDC]
	(No symbol) [0x00007FF66C545C64]
	GetHandleVerifier [0x00007FF66CA6E16B+3695259]
	GetHandleVerifier [0x00007FF66CAC6737+4057191]
	GetHandleVerifier [0x00007FF66CABE4E3+4023827]
	GetHandleVerifier [0x00007FF66C7904F9+689705]
	(No symbol) [0x00007FF66C66C048]
	(No symbol) [0x00007FF66C668044]
	(No symbol) [0x00007FF66C6681C9]
	(No s

...


I am BLOCKED
Waiting 120 seconds ...

                              

I am BLOCKED
Waiting 240 seconds ...

                              

I am BLOCKED
Waiting 480 seconds ...

                              

I am BLOCKED
Waiting 960 seconds ...

                              
189- Scraped and Saved Trompette 8.
190- Scraped and Saved No. 03 L’Eau Ivre (Interdit).
191- Scraped and Saved Flowerbomb La Vie en Rose 2013.
192- Scraped and Saved Atlante.
193- Scraped and Saved Bois.
194- Scraped and Saved Ameer Al Oudh.
195- Scraped and Saved Paradize.
196- Scraped and Saved New York Patchouli.
197- Scraped and Saved Holy Neroli.

I am BLOCKED
Waiting 120 seconds ...

                              
198- Scraped and Saved Good Girl Superstars Collector Edition 2021.
199- Scraped and Saved Paradiso Assoluto.
200- Scraped and Saved Good Vibes.
201- Scraped and Saved Si Rose Signature.
202- Scraped and Saved Messy Sexy Just Rolled out of Bed.
203- Scraped and Saved Arancia Rossa.
204- Scraped

341- Scraped and Saved Diesel Green Masculine Special Edition.
342- Scraped and Saved Adeeb.

I am BLOCKED
Waiting 120 seconds ...

                              
343- Scraped and Saved Flowerbomb.
344- Scraped and Saved Fleurs d'Ombre Nymphea.
345- Scraped and Saved Aqua Allegoria Passiflora.
346- Scraped and Saved Ouverture.
347- Scraped and Saved Ett Hem.
348- Scraped and Saved Tabac Libre.
349- Scraped and Saved Romance Parfum.
350- Scraped and Saved 4.20 PM.
351- Scraped and Saved Tharwah Gold.

I am BLOCKED
Waiting 120 seconds ...

no elements could be found with tag_name "body"
352- Scraped and Saved Journey Man.
353- Scraped and Saved Gloam.
354- Scraped and Saved Sugar.
355- Scraped and Saved Ilio.
356- Scraped and Saved Hit Orange.
357- Scraped and Saved Hasunoito.
358- Scraped and Saved Tuscan Soul.
359- Scraped and Saved Mexx Waterlove Man.
360- Scraped and Saved Chrome Eau de Parfum.

I am BLOCKED
Waiting 120 seconds ...

                              

I am BLOCKED
Waitin

496- Scraped and Saved Physical Man.
497- Scraped and Saved Ultraviolet Liquid Crystal Man.
498- Scraped and Saved Black Orchid Voile de Fleur.
499- Scraped and Saved Yujin Pour Homme.

I am BLOCKED
Waiting 120 seconds ...

                              
500- Scraped and Saved U BLUE Women.
501- Scraped and Saved Paper Flower.
502- Scraped and Saved Kuwait.
503- Scraped and Saved Want.
504- Scraped and Saved Golden Eyes.
505- Scraped and Saved Flowerhead.
506- Scraped and Saved Oligarch.
507- Scraped and Saved Alien Man Fusion.
508- Scraped and Saved Bonbon Pop.

I am BLOCKED
Waiting 120 seconds ...

                              
509- Scraped and Saved Velvet Orchid Lumière.
510- Scraped and Saved Maritime for Him.
511- Scraped and Saved Constantinople.
512- Scraped and Saved Emporio Armani Stronger With You Leather.
513- Scraped and Saved Supergloss.
514- Scraped and Saved Io Non Ho Mani Che Mi Accarezzino il Volto.
515- Scraped and Saved Florence.
516- Scraped and Saved Bloody Rose.

659- Scraped and Saved Tuesday - Totally Terrific.
660- Scraped and Saved Le Bleu.
661- Scraped and Saved Jersey Eau de Parfum.
662- Scraped and Saved Bandit Suprême.
663- Scraped and Saved Cubia.
664- Scraped and Saved Solo Atlas.

I am BLOCKED
Waiting 120 seconds ...

                              
665- Scraped and Saved Cococabana.
666- Scraped and Saved Aspen Caramel Woods.
667- Scraped and Saved Gabrielle.
668- Scraped and Saved Kashmina Touch.
669- Scraped and Saved L'Eau du Temps.
670- Scraped and Saved 702 encens, lavande, bois de cachemire.
671- Scraped and Saved Opulent Red.
672- Scraped and Saved Flower Fusion.
673- Scraped and Saved Pure Extreme Nectar.

I am BLOCKED
Waiting 120 seconds ...

                              
674- Scraped and Saved Black Vanilla Absolute.
675- Scraped and Saved Fleur Monoï.
676- Scraped and Saved Cassandra Blanc.
677- Scraped and Saved Myrrh.
678- Scraped and Saved Esco Pazzo .
679- Scraped and Saved Florentine Iris Eau de Parfum.
680- Scraped 

                              

I am BLOCKED
Waiting 480 seconds ...

                              

I am BLOCKED
Waiting 960 seconds ...

                              
                              
817- Scraped and Saved Tabac Sport.
818- Scraped and Saved Puma Create Man.
819- Scraped and Saved Vive la Mariée.
820- Scraped and Saved Le Cri de la Lumière.
821- Scraped and Saved Infusion de Rose (2017).
822- Scraped and Saved Oriental.
823- Scraped and Saved Bvlgari Man Extrême All Black Editions.
824- Scraped and Saved Dianoche Day.

I am BLOCKED
Waiting 120 seconds ...

                              
825- Scraped and Saved Giardinodiercole.
826- Scraped and Saved Mica D'Oro.
827- Scraped and Saved La Selection Parfum Sacre.
828- Scraped and Saved Sacrebleu Intense.
829- Scraped and Saved Lilacs & Gooseberries.
830- Scraped and Saved Ambre Platine.
831- Scraped and Saved Alibi.
832- Scraped and Saved Macaque Fuji Apple Edition.
833- Scraped and Saved Ambrosia Imperiale.

I am BLOCK

In [ ]:
# from fp.fp import FreeProxy
# from fp.errors import FreeProxyException

In [ ]:
# url = "https://www.fragrantica.com/perfume/Maison-Francis-Kurkdjian/Baccarat-Rouge-540-33519.html"

In [ ]:
# while True:
#     try:
#         proxy = FreeProxy(https=True).get()
#         print(f"New Proxy: {proxy}")
        
#     except FreeProxyException as e:
#         print(e)
#         continue

#     browser = init_browser(proxy)
#     browser.visit(url)

#     if is_cloudflared(browser) or is_disconnected(browser):
#         print("connection Failed ...\n")
#         browser.quit()
#     else:
#         print("Sustainable Connection FOUND. Hoooray.")
#         break